In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch import autograd
from torch.nn import init
import math
from ops import mu_law_encode, one_hot, time_to_batch, batch_to_time, mu_law_decode
from model import WaveNetModel
from audio_reader import AudioReader
import torch.optim as optim
import librosa
import time
import queue as Queue
import json
FILE_PATTERN = r'p([0-9]+)_([0-9]+)\.wav'

ImportError: No module named queue

In [ ]:
with open('./wavenet_params.json', 'r') as config_file:
    wavenet_params = json.load(config_file)

In [ ]:
model = WaveNetModel(
                 wavenet_params["batch_size"],
                 wavenet_params["dilations"],
                 wavenet_params["filter_width"],
                 wavenet_params["residual_channels"],
                 wavenet_params["dilation_channels"],
                 wavenet_params["skip_channels"],
                 quantization_channels = wavenet_params["quantization_channels"],
                 use_biases = wavenet_params["use_biases"],
                 scalar_input = wavenet_params["scalar_input"],
                 use_cuda = wavenet_params["use_cuda"],
                 initial_filter_width = wavenet_params["initial_filter_width"],
                 global_condition_channels=None,
                 global_condition_cardinality=None)
if model.use_cuda:
    model = model.cuda()
model.load_state_dict(torch.load('/home/administrator/workspace/true_model/logdir/epoch300-loss=1.773_model.txt'))

In [5]:
num_iters = 10000
waveform = [model.quantization_channels / 2] * (model.receptive_field - 1)
waveform.append(np.random.randint(model.quantization_channels))
for i, x in enumerate(waveform):
    if model.use_cuda:    
        loss = model.predict_proba_incremental(autograd.Variable(torch.FloatTensor([x])).cuda())
    else:
        loss = model.predict_proba_incremental(autograd.Variable(torch.FloatTensor([x])))   
start_time = time.time()
waveform = [waveform[-1]]
for it in range(num_iters):
    model.zero_grad()
    apmlitude = waveform[-1]
    if model.use_cuda:    
        loss = model.predict_proba_incremental(autograd.Variable(torch.FloatTensor([apmlitude.astype(float)])).cuda())
    else:
        loss = model.predict_proba_incremental(autograd.Variable(torch.FloatTensor([apmlitude.astype(float)])))
    if model.use_cuda:  
        index = np.random.choice(np.arange(model.quantization_channels),p=loss.cpu().data.numpy())
    else:
        index = np.random.choice(np.arange(model.quantization_channels),p=loss.data.numpy())
    #_, index = torch.max(loss,1)
    #index = index.cpu().data.numpy()[0]
    if it % 100 == 0:
        print (it,' ',index)
        print (time.time() - start_time)
    waveform.append(index)


0   231
0.12192130088806152


RuntimeError: tried to construct a tensor from a float sequence, but found an item of type numpy.int64 at index (0)

In [18]:
waveform = [waveform[-1]]
for it in range(num_iters):
    model.zero_grad()
    apmlitude = waveform[-1]
    if model.use_cuda:    
        loss = model.predict_proba_incremental(autograd.Variable(torch.FloatTensor([apmlitude.astype(float)])).cuda())
    else:
        loss = model.predict_proba_incremental(autograd.Variable(torch.FloatTensor([apmlitude.astype(float)])))
    if model.use_cuda:  
        index = np.random.choice(np.arange(model.quantization_channels),p=loss.cpu().data.numpy())
    else:
        index = np.random.choice(np.arange(model.quantization_channels),p=loss.data.numpy())
    #_, index = torch.max(loss,1)
    #index = index.cpu().data.numpy()[0]
    if it % 100 == 0:
        print (it,' ',index)
        print (time.time() - start_time)
    waveform.append(index.astype(float))

0   68
590.9403784275055
100   5
595.8926148414612
200   23
600.1210551261902
300   48
604.8428466320038
400   92
608.4205298423767
500   50
613.4706163406372
600   153
618.3117105960846
700   126
621.7160153388977
800   195
625.2276086807251
900   28
630.2861328125
1000   185
633.8129954338074
1100   227
639.0368657112122
1200   79
642.9706246852875
1300   67
647.7975232601166
1400   65
651.53937458992
1500   62
655.1185615062714
1600   75
660.8131058216095
1700   62
665.7906303405762
1800   218
671.3805019855499
1900   64
675.4747653007507
2000   25
679.3875186443329
2100   183
684.8002252578735
2200   46
688.5876297950745
2300   175
692.4144303798676
2400   88
697.6148805618286
2500   37
701.5752880573273
2600   4
705.3892149925232
2700   224
711.5889971256256
2800   73
716.7163963317871
2900   68
723.4136481285095
3000   12
727.636075258255
3100   45
732.4722464084625
3200   164
738.8399014472961
3300   56
743.4542207717896
3400   53
748.7725694179535
3500   228
754.9056000709534
3

KeyboardInterrupt: 

In [19]:
to_save = autograd.Variable(torch.FloatTensor(waveform))
to_save = mu_law_decode(to_save,256)
librosa.output.write_wav('new_gen.wav', to_save.data.numpy(), 2000)


In [11]:
apmlitude.astype(float).dtype

dtype('float64')